In [ ]:
!pip install transformers datasets torch scikit-learn pandas


In [11]:
# Import libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [12]:
import pandas as pd

# Load the dataset
df = pd.read_csv('AllProductReviews.csv')

# Frustration keywords
frustration_keywords = [
    "waste", "useless", "not working", "stopped", "disappointed", "poor", "frustrating",
    "doesn't work", "irritating", "worst", "problem", "issue", "disconnect", "bad", "low quality",
    "not worthy", "not worth", "unworthy", "stops", "noisy", "crash", "lag", "uncomfortable", "unreliable",
    "not reliable", "cracked", "crack", "isn't working"
]

# Labeling function
def label_frustration(text):
    text = str(text).lower()
    return int(any(kw in text for kw in frustration_keywords))

# Apply labeling
df['Frustration_Label'] = df['ReviewBody'].apply(label_frustration)

# Prepare final dataset
df = df[['ReviewBody', 'Frustration_Label']].dropna()
df = df.rename(columns={'ReviewBody': 'text', 'Frustration_Label': 'label'})
df.head()


,text,label
0,No doubt it has a great bass and to a great ex...,1
1,"This earphones are unreliable, i bought it be...",1
2,"i bought itfor 999,I purchased it second time,...",0
3,Its sound quality is adorable. overall it was ...,1
4,Its Awesome... Good sound quality & 8-9 hrs ba...,1


In [13]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Save to CSV
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)


In [17]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Choose a small model (prajjwal1/bert-mini is lighter than DistilBERT)
model_name = "prajjwal1/bert-mini"  # You can also try: "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load dataset
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

# Tokenize text
def tokenize_function(example):
    return tokenizer(example["text"], 
                     padding="max_length", 
                     truncation=True,
                    max_length = 512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


C:\Users\Adhar\anaconda3\envs\tf_env\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|████████████████████████████████████████████████████████████████| 2868/2868 [00:00<00:00, 4906.21 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
df['token_count'] = df['text'].apply(lambda x: len(tokenizer.tokenize(str(x))))
df[df['token_count'] > 512].shape

(4, 3)

In [19]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Define metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_strategy="epoch",
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"], 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\Adhar\anaconda3\envs\tf_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.181900,0.072142,0.984310,0.963277,0.972896,0.968062
2,0.051700,0.036603,0.992678,0.984330,0.985735,0.985032
3,0.032200,0.028471,0.994073,0.989971,0.985735,0.987848


Checkpoint destination directory ./results\checkpoint-1434 already exists and is non-empty.Saving will proceed but saved results may be invalid.
C:\Users\Adhar\anaconda3\envs\tf_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Checkpoint destination directory ./results\checkpoint-2868 already exists and is non-empty.Saving will proceed but saved results may be invalid.
C:\Users\Adhar\anaconda3\envs\tf_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=4302, training_loss=0.08857486925253698, metrics={'train_runtime': 4263.1087, 'train_samples_per_second': 8.071, 'train_steps_per_second': 1.009, 'total_flos': 340967719544832.0, 'train_loss': 0.08857486925253698, 'epoch': 3.0})

In [21]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json',
 './saved_model\\tokenizer.json')

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")


In [24]:
# Create a pipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False)

# Test on new reviews
sample_reviews = [
    "This product is useless and keeps disconnecting. Very disappointed.",
    "Absolutely love this earphone. Great quality and battery life!",
    "Stopped working after 2 days. Waste of money!"
]

for review in sample_reviews:
    result = pipe(review)
    label = result[0]['label']
    score = result[0]['score']
    print(f"Review: {review}\n → Prediction: {label} (Confidence: {score:.2f})\n")


Review: This product is useless and keeps disconnecting. Very disappointed.
 → Prediction: LABEL_1 (Confidence: 1.00)

Review: Absolutely love this earphone. Great quality and battery life!
 → Prediction: LABEL_0 (Confidence: 1.00)

Review: Stopped working after 2 days. Waste of money!
 → Prediction: LABEL_1 (Confidence: 1.00)



C:\Users\Adhar\anaconda3\envs\tf_env\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
